# Annotation der Test Daten

Hier sollen zu einigen Passagen / Dokumenten Fragen formuliert und die Antworten dazu annotiert werden. Dabei soll dem SQUAD-Standard gefolgt werden:

| ID (int) | context (string) | question (string) | answers (json) |
| -: | :- | :- | :- |
| 1 | In der letzten Spalte der Signalanbindungstabelle kann eine Standardeinheit für ein generisches Signal angegeben werden. Der Defaulteintrag don’t care bezeichnet die Übernahme der Quelleinheit aus der Aufnahmedatei. | Wo kann die Einheit für genererisches Signal angegeben werden? | { "text": [In der letzten Spalte der Signalanbindungstabelle], "answer_start": [1] } |
| ... | ... | ... | ... | ... |

Dabei muss sich die ID dem Kontext in der geparsden Doku zuzuordenen sein. Für einen Kontext können mehrere Fragen formuliert werden, auch können mehrere Antworten gegeben werden (Im SQUAD ist der Standard 3, wobei eine Antwort (Antwort-Text und Start-Token) dann wiederholt wird, wenn es im gegeben Kontext nur eine Stelle mit der passenden Antwort gibt, so dass immer drei Antworten pro Fragen annotiert sind). Es sollte ausreichend sein, sich hier auf eine Antwort pro Frage zu beschränken. Für die Ermittlung des Start-Tokens werden Buchstaben, keine Wörter gezählt.

Die Idee ist ein zufälliges Dokument auszuwählen und dann eine oder mehrer Fragen zu formulieren. Die Antwort kann dann aus dem Text herausgesucht werden. Über Rückwärtssuche im gegebenen Text / Kontext, kann dann die Antwort im json-Format automatisch erzeugt werden.

### Laden der Dokumente

In [1]:
documents = catalog.load("ecu_test_doku_parsed")

2022-05-10 20:03:39,910 - kedro.io.data_catalog - INFO - Loading data from `ecu_test_doku_parsed` (CSVDataSet)...


### Definieren der nötigen Funtkionen

In [74]:
import pandas as pd

In [108]:
def get_random_document():
    _id = documents.sample()['body'].index[0]
    context = str(documents.sample()['body'].iloc[0])
    return _id, context

def get_start_token_of_string_in_context(context, query_string):
    res = context.find(query_string)
    if res == -1:
        print("context does not contain string")
        return {}
    else:
        return {"text": query_string, "answer_start": res}

### Beispiel Anwendung

In [90]:
# define output
answers = []

#### Wahl eines zufälligen Dokumentes

In [103]:
_id, context = get_random_document()
print(f'Picked Document with id: {_id}, context is: {context[:25]}...')

Picked Document with id: 438, context is: Port ENVSIM ? — ECU-TEST ...


#### DEBUG: Überschreiben des generierten Kontextes, da html Parser noch nicht geupdatet.

In [105]:
context = "Das ist ein Dummytext. In der echten Anwendung würde hier ein Dokument stehen."
context

'Das ist ein Dummytext. In der echten Anwendung würde hier ein Dokument stehen.'

#### Stellen und Beantworte der Frage

In [107]:
question = "Was würde hier in der echten Anwendung stehen?"
answer = "ein Dokument"

answer_json = get_start_token_of_string_in_context(context, answer)
answers.append([_id, context, question, answer])

### Überführen der Anworten in DataFrame und Speichern als CSV

In [94]:
answers_df = pd.DataFrame(answers)
answers_df.columns = ['id', 'context', 'question', 'answers']
display(answers_df)

,id,context,question,answers
0,362,Das ist ein Dummytext. In der echten Anwendung...,Was würde hier in der echten Anwendung stehen?,"{'text': 'ein Dokument', 'answer_start': 58}"


In [109]:
answers_df.to_csv("../data/03_primary/test_data.csv")